## BOAR with SIMsalabim
Version 0.1
(c) Larry Lueer, Vincent M. Le Corre, i-MEET 2021-2023

This notebook is made to use BOAR in combination with drift-diffusion modeling to fit of experimental JV curves.\
To perform the drift-diffusion simulation in the background we use the open-source program [SIMsalabim](https://github.com/kostergroup/SIMsalabim), for more information about SIMsalabim please check the [GitHub repository](https://github.com/kostergroup/SIMsalabim) 

Here we are fitting an typical OPV cell with the following structure:\
ITO|ZnO|Active layer|MoOx|Ag

In [ ]:
# Activate matplotlib widgets
%matplotlib inline
# comment the next line if you are on the jupyterhub server
%matplotlib widget 
# %matplotlib notebook

# Import libraries
import sys
from numpy.random import default_rng
from scipy import interpolate
from sklearn.preprocessing import minmax_scale
from tabulate import tabulate
from datetime import datetime
import warnings
warnings.filterwarnings('ignore') # comment this out to see warnings

# Import boar
# import one folder up
# sys.path.append('../') # comment out if the Notebook is in the Notebooks folder
from boar import *
from boar.SIMsalabim_utils.MakeDevParFile import *
from boar.SIMsalabim_utils.GetInputPar import *
from boar.SIMsalabim_utils.PlotInputPar import *
# Import homemade package by VLC
# import boar.SIMsalabim_utils.plot_settings_screen # to set default plot settings

# Import imeetDB
sys.path.append(os.path.join('/home/vlc/Desktop/','imeetdb')) #  absolute path to imeetDB
# Import local libraries
from imeetdb.dbconnect import *
from imeetdb.models import *
from agents.imeetDB_JV_agent import *

# Import the database session
db_session

# Import slack utils
sys.path.append(os.path.join('/home/vlc/Desktop/','boar_slack_bot')) #  absolute path to imeetDB
from boar_slack_bot.slack_utils import *

# Import Experimental Data

In [ ]:
# Define the path to the data and to the SIMsalabim directory
curr_dir = os.getcwd()

# Define the path to the SIMsalabim directory and the results directory
path2simu = os.path.join('/home/vlc/Desktop/Xioyan','SIMsalabimv445_Xioyan','SimSS') # path to the SIMsalabim directory
res_dir = os.path.join(curr_dir,'temp') # path to the results directory, use this line if Notebook is in the main folder
# res_dir = os.path.join('../','temp') # path to the results directory, use this line if Notebook is in the Notebooks folder



## Select device from the i-MEET database

In [ ]:
# Select username and PID of the sample
user = 'osterriedert'
PID = '20517'
pixel = 'B-1'
time_date = '2023-03-28 11:36:51'
#convert time_date to datetime object
time_date = datetime.strptime(time_date, '%Y-%m-%d %H:%M:%S')
area = 0.08 *1e-4 # cm^2 * 1e-4 to get m^2


### Check the stack of the device

In [ ]:
# test get stack info and process conditions
stack_info = get_stack_info_from_db(user,PID)

active_layer = []
for S0 in stack_info:
    print(S0)
    if S0['Type'] == 'active':
        for key in S0.keys():
            # check if key start with M
            if key[0] == 'M' and S0[key] != '':
                active_layer.append(S0[key])

print('Active layer:')
print(active_layer)

CB_vals = []
VB_vals = []

# full acceptor list
acceptor_list = ['Y6','Y6-BO-4F','DT-Y6','BTP-4F-12','Y12','ITIC-4F','ITIC-Th','ITIC','EH-IDTBR','O-IDTBR','O-IDFBR','IEICO-4F','PC71BM']
CBs = [4.07,4.04,4.04,4.06,4.06,4.1,3.75,3.81,3.81,3.88,3.89,4.19,4.12]

# full donor list
donor_list = ['PM6','PTQ10','PCE10','PffBT4T-2DT','D18','PDPP-5T','D4610','PTB7']
VBs = [5.5,5.55,5.23,5.24,5.56,5.14,5.2,5.15]


for i in active_layer:
    if i in acceptor_list:
        CB_vals.append(CBs[acceptor_list.index(i)])
    elif i in donor_list:
        VB_vals.append(VBs[donor_list.index(i)])
    else:
        pass

CB_val = max(CB_vals)
VB_val = min(VB_vals)
print( 'CB = ', CB_val, 'eV')
print( 'VB = ', VB_val, 'eV')

### Check the structure of the device for SIMsalabim
Here we check the structure of the device for SIMsalabim and you can adjust the parameters if needed.

For example, if you want to change the thickness of the active layer (AL) you can do this by changing the value of the variable `L` in the cell below.
To do so you need to update the value of the variable ParFileDic['L'] and set MakeUpdate to True.

By convention LTL is the ETL and RTL is the HTL.\
Note that L is the **total** thickness of the ETL, HTL and active layer, so if you want to change the thickness of the active layer you need to subtract the thickness of the ETL and HTL from the total thickness. 
Such that:

$L_{AL} = L - L_{LTL} - L_{RTL}$

To check the name of the parameters in SIMsalabim please refer to the [SIMsalabim documentation](https://github.com/kostergroup/SIMsalabim).



In [ ]:
# Vizualize the stack defined for the simulation
check_SIMsalabim_input = False
if check_SIMsalabim_input:
    # Load Device parameters file and update parameters
    ParFileDic = ReadParameterFile(os.path.join(path2simu, 'device_parameters.txt')) # read the parameters from the file

    # Change parameters in the dictionary
    ParFileDic['CB'] = CB_val
    ParFileDic['VB'] = VB_val
    ParFileDic['CB_LTL'] = CB_val
    ParFileDic['W_L'] = CB_val
    ParFileDic['W_R'] = VB_val
    MakeUpdate = False # set to True to update the parameters in the file
    if MakeUpdate:
        UpdateDevParFile(ParFileDic, path2simu) # update the parameters in the file

    # Visualize the parameters
    fig, axs = plt.subplots(2,2,figsize = (10,8))
    plot_input_nrj_diag(ParFileDic,ax=axs[0, 0])
    plot_input_mob(ParFileDic,ax=axs[0, 1])
    plot_input_dens(ParFileDic,ax=axs[1, 0])
    plot_input_SRH_lifetime(ParFileDic,ax=axs[1, 1],y_unit='ns',y2_unit='cm')
    plt.tight_layout()
    plt.show()


In [ ]:
# Define function to get JV from imeetDB
def filter_DB_data(username,PID,pixel=None,time2fit=None,Gfracs=[1],area=0.08 *1e-4,Vlim=[-0.5,1],time_format='relative',time_unit='h'):
    """ Get the JVs from imeetDB and calculate some weights for the JV

    Parameters
    ----------
    username : str
        username of the user
    PID : str
        PID of the sample
    pixel : str, optional
        pixel of the sample, by default None
    time2fit : datetime or float
        time of the measurement. If time_format is 'absolute' then time is a datetime object. If time_format is 'relative' then time is a float.
    Gfracs : list, optional
        list of light intensities in suns, by default [1]
    area : float, optional
        area of the sample in m^2, by default 8e-6
    Vlim : list, optional
        voltage limits for the JV, by default [-0.5,1]
    time_format : str, optional
        time format, by default 'absolute'
        'absolute' : time is a datetime object
        'relative' : time is a float
    time_unit : str, optional
        time unit, by default 'min'
        'min' : time is in minutes
        'h' : time is in hours
        's' : time is in seconds

    Returns
    -------
    X : array
        array of the voltage and light intensity
    y : array
        array of the current
    weight : array
        array of the weights

    """    

    if pixel is None:
        pixel = get_champion_pixel_from_db(username,PID)

    if time2fit is None: # get the JV at t=0
        X,y = get_JV_from_db(username,PID,pixel,0,Gfracs=Gfracs,area=area,time_format='relative',time_unit='h')
    else:
        X,y = get_JV_from_db(username,PID,pixel,time2fit,Gfracs=Gfracs,area=area,time_format=time_format,time_unit=time_unit)

    # filter data
    idx = np.where((X[:,0] > Vlim[0]) & (X[:,0] < Vlim[1]))[0]
    X = X[idx,:]
    y = y[idx]

    # get unique values of Gfrac
    Gfrac_unique = np.unique(X[:,1])
    weight = np.ones(len(y))
    for Gfrac in Gfrac_unique:
        idx = np.where(X[:,1] == Gfrac)[0]
        V = X[idx,0]
        J = y[idx]

        if Gfrac == 0: # dark so weight = 1
            weight[idx] = 1
        else:
            # weight[idx] = 1/np.sqrt(np.max(J)-np.min(J))

            power = -np.array(V)*np.array(J)
            power = minmax_scale(np.asarray(power), feature_range=(1, 100))
            weight[idx] = power
        

    return X,y,weight

X,y,weight = filter_DB_data(user,PID,pixel=None,time2fit=time_date,Vlim=[-0.5,1],Gfracs=[0.1,0.5,1,1.2],area=area,time_format='absolute')

f = plt.figure()
plt.subplot(1,2,1)
plt.plot(X[:,0],y,'o')
plt.xlabel('Voltage [V]')
plt.ylabel('Current density [A m$^2$]')
plt.ylim([min(y)*1.1,10])

plt.subplot(1,2,2)
plt.plot(X[:,0],weight,'o')
plt.xlabel('Voltage [V]')
plt.ylabel('Weight')
plt.tight_layout()
plt.show()

# Fit non-ideal diode model
In this section you need to write a function that fits the non-ideal diode model to dark-JV curves. We do this to get the series resistance ('Rseries') and shunt resistance ('Rshunt') from the dark-JV curves to avoid fitting them in the multi-objective optimization. 

Note: Running this cell is not necessary, run it only if you want to see the results.

In [ ]:
# Import  and fit diode equation to dark JV data
fit_dark = True
colors = ['b','g','r','c','m','y','k']
if fit_dark:
    markers = ['o','s','^','v']
    Rs, Rsh, J0, n, errRs, errRsh, errJ0, errn = [],[],[],[],[],[],[],[]
    idx = 0 
    dio = Non_Ideal_Diode_agent()
    plt.figure(12)
        
    X,y,weights = filter_DB_data(user,PID,pixel=None,Vlim=[1e-2,3],Gfracs=[0],area=area)

    V = X[:,0]
    J = y
    #remove values with J = 0
    V = V[J!=0]
    J = J[J!=0]
    # Fit the non ideal diode equation
    res = dio.FitNonIdealDiode(V,J,T=300,JV_type='dark',take_log=True,bounds=([1e-12, 1, 1e-6, 1e-2], [1e-3, 3, 1e-2, 1e4]),p_start={'J0':1e-8})

    Rs.append(res['Rs'])
    Rsh.append(res['Rsh'])
    J0.append(res['J0'])
    n.append(res['n'])
    errRs.append(res['Rs_err'])
    errRsh.append(res['Rsh_err'])
    errJ0.append(res['J0_err'])
    errn.append(res['n_err'])

    plt.semilogy(V,abs(J),'o',c='C'+str(idx))
    plt.semilogy(V,abs(dio.NonIdealDiode_dark(V,res['J0'],res['n'],res['Rs'],res['Rsh'])),c='C'+str(idx))
    


    plt.xlabel('Voltage [V]')
    plt.ylabel('Current [A/m$^2$]')
    plt.ylim(1e-4,1e5)
    plt.legend()
    plt.show()

    # Print the results
    print('Rs = ',Rs)
    print('Rsh = ',Rsh)
    print('J0 = ',J0)
    print('n = ',n)

    plt.tight_layout()
    plt.show()

# Multi-Objective Optimization
Below we will perform the Bayesian optimization to fit the experimental data.

### Define the fitting parameters

In [ ]:
# define Fitparameters
Start_values = {'kdirect':5e-18,'mun_0':2e-8,'mup_0':8e-8,'Nc':5e26,'Gehp':1.28e28,'Bulk_tr':1e20,'Gehp':1.28e28}
params = []

Bulk_tr = Fitparam(name = 'Bulk_tr', val = Start_values['Bulk_tr'] , relRange = 0.2, lims=[1e16,1e21],range_type='log',optim_type='log',lim_type='absolute',display_name='N$_{Tr}$ [m$^{-3}$]') #real 6.4e18 
params.append(Bulk_tr)

kdirect = Fitparam(name = 'kdirect', val = Start_values['kdirect'] , relRange = 0.2, lims=[1e-20,1e-16],range_type='log',optim_type='log',lim_type='absolute',display_name='k$_{2}$ [m$^{3}$ s$^{-1}$]')
params.append(kdirect)

mun_0 = Fitparam(name = 'mun_0', val = Start_values['mun_0'] , relRange = 0.2, lims=[5e-9,1e-6],range_type='log',optim_type='log',lim_type='absolute',display_name='$\mu_n$ [m$^{2}$ V$^{-1}$ s$^{-1}$]')
params.append(mun_0)

mup_0 = Fitparam(name = 'mup_0', val = Start_values['mup_0'] , relRange = 1.5, lims=[5e-9,1e-6],range_type='log',optim_type='log',display_name='$\mu_p$ [m$^{2}$ V$^{-1}$ s$^{-1}$]')
params.append(mup_0)

Nc = Fitparam(name = 'Nc', val = Start_values['Nc'] , relRange = 1, lims=[0.5e26,5e27],range_type='log',optim_type='log',lim_type='absolute',display_name='N$_{c}$ [m$^{-3}]$')
params.append(Nc)

St_L = Fitparam(name = 'St_L', val = 1e11 , relRange = 0.2, lims=[1e10,1e15],range_type='log',optim_type='log',lim_type='absolute',display_name='S$_{Tr}^{ETL}$ [m$^{-2}$]')
params.append(St_L)

Rseries = Fitparam(name = 'Rseries', val = 1e-4, relRange = 1, lims=[1e-6,1e-2],range_type='log',optim_type='log',lim_type='absolute',display_name='R$_{series}$ [$\Omega$ m$^2$]')
params.append(Rseries)

Rshunt = Fitparam(name = 'Rshunt', val = 3e2 , relRange = 1, lims=[1e-1,1e3],range_type='log',optim_type='log',lim_type='absolute',display_name='R$_{shunt}$ [$\Omega$ m$^2$]')
params.append(Rshunt)

W_R = Fitparam(name = 'W_R', val = 0 , relRange = 1, lims=[-0.4,0],range_type='linear',optim_type='linear',lim_type='absolute',display_name='W$_R$ [eV]')
params.append(W_R)

W_L = Fitparam(name = 'W_L', val = 0 , relRange = 0, lims=[0,0.2],range_type='linear',optim_type='linear',lim_type='absolute',display_name='W$_L$ [eV]')
params.append(W_L)

CB_LTL = Fitparam(name = 'CB_LTL', val = 0 , relRange = 1, lims=[0,0.35],range_type='linear',optim_type='linear',lim_type='absolute',display_name='CB$_{ETL}$ [eV]')
params.append(CB_LTL)


Gehp = Fitparam(name = 'Gehp', val = Start_values['Gehp'] , relRange = 0.2, lims=[1.45e27,1.75e28],range_type='log',optim_type='linear',axis_type='log',lim_type='absolute',display_name='G$_{ehp}$ [m$^{-3}$ s$^{-1}$]')
params.append(Gehp)

ETrapSingle = Fitparam(name = 'ETrapSingle', val = 0.157 , relRange = 1, lims=[0.1,0.7],range_type='linear',optim_type='linear',lim_type='absolute',display_name='E$_{Trap}$ [eV]')
params.append(ETrapSingle)

# Energy levels
CB = Fitparam(name = 'CB', val = CB_val , relRange = 0, lims=[3.6,4.1],range_type='linear',optim_type='linear',lim_type='absolute',display_name='CB [eV]')
params.append(CB)

VB = Fitparam(name = 'VB', val = VB_val , relRange = 0, lims=[0.5,0.8],range_type='linear',optim_type='linear',lim_type='absolute',display_name='VB [eV]')
params.append(VB)



params_true = copy.deepcopy(params)


### Start the optimization

In [ ]:
# Get experimental data light JV
X,y,weights = filter_DB_data(user,PID,pixel=None,time2fit=time_date,Vlim=[-0.5,0.9],Gfracs=[0.1,0.5,1,1.2],area=area,time_format='absolute')
X_dimensions = ['Vext','Gfrac']

# Get Vmax and Vmin
Vmax = np.around(np.max(X[:,0]),2)
Vmin = np.min(X[:,0])

# Update the device_parameters.txt file with the new thickness
ParFileDic = ReadParameterFile(os.path.join(path2simu, 'device_parameters.txt')) # read the parameters from the file


fixed_str = '-Vmin {:.3f} -Vmax {:.3f} '.format(Vmin-0.05,Vmax+0.05) # string to be added to the command line call to fix the Vmin and Vmax and some numerical parameters to help/speed up the simulation

# Define weighting for the different JV curves
use_weighting = True
if use_weighting:
    weight = weights
else:
    weight = 1

# initialize the simulation agent
dda = Drift_diffusion_agent(path2simu=path2simu) 

NewName = ''
target = {'model':partial(dda.DriftDiffusion_relative,X_dimensions=X_dimensions,max_jobs=4,fixed_str=fixed_str,dev_par_fname=NewName),'target_name':'JV'
          ,'data':{'X':X,'y':y,'X_dimensions':['Vext','Gfrac'],'X_units':['V','sun'],'y_dimension':'Current density','y_unit':r'$A m^{-2}$'}
           , 'weight':weight,'target_weight':1}

# Define optimizer
mo = MultiObjectiveOptimizer(res_dir=res_dir,params=params,targets=[target],SaveOldXY2file=os.path.join(res_dir,'XY_old.json'),Path2OldXY=os.path.join(res_dir,'XY_old.json')) # initialize the optimizer
mo.warmstart = 'collect_init' # 'recall' data from Path2OldXY file

# Define the number of iterations for the optimization
n_jobs = 4
n_jobs_init = 20
n_yscale=20
n_initial_points = 80
n_BO = 20
n_BO_warmstart = 80

kwargs = {'check_improvement':'relax','max_loop_no_improvement':10,'xtol':1e-3,'ftol':1e-3}
kwargs_posterior = {'Nres':4,'gaussfilt':1,'logscale':False,'vmin':1e-100,'zoom':0,'min_prob':1e-40,'clear_axis':True,'show_points':True,'savefig':True,'figname':'param_posterior' ,'show_fig':True,'figsize':(14,14)}
kwargs_plot_obj = {'zscale':'linear','show_fig':False}

r = mo.optimize_sko_parallel(n_jobs=n_jobs,n_yscale=n_yscale, n_BO=n_BO, n_initial_points = n_initial_points,n_BO_warmstart=n_BO_warmstart,n_jobs_init=n_jobs_init,kwargs=kwargs,verbose=False,loss='cauchy',threshold=50,base_estimator = 'GP',show_objective_func=False,show_posterior=True,kwargs_posterior = kwargs_posterior,kwargs_plot_obj=kwargs_plot_obj)
# pf.append(deepcopy(target['params'])) # collects optimized fitparameters
rrr = r['r'] # the results dict of the last optimizer.tell()

best_params = copy.deepcopy(mo.params) # get the best parameters

In [ ]:
# Plot and save the results
kwargs_JV_plot = {'savefig':True,'figname':'fits_JV','figdir':res_dir,'show_fig':True,'figsize':(10,8)}
dda.Compare_JVs_exp(target,best_params,FOMs=[],verbose=True,ylim=[],fixed_str=NewName,DD_func=partial(dda.DriftDiffusion_relative,X_dimensions=X_dimensions,max_jobs=4,fixed_str=fixed_str,dev_par_fname=NewName),kwargs=kwargs_JV_plot) #-100,10

Jfit = dda.DriftDiffusion_relative(X,best_params,X_dimensions=X_dimensions,max_jobs=4,fixed_str=fixed_str,dev_par_fname=NewName)

data = np.concatenate((X, y.reshape(len(y),1), Jfit.reshape(len(Jfit),1)), axis=1)

# prepare the data for saving
param_dict = dda.get_param_dict(best_params) # get fitparameters (and fixed ones)
pout = [[f'{v:.3E}' if isinstance(v,float) else v for _,v in pp.items()] for pp in param_dict]

# plot the fitparameters
kwargs_plot_params = {'nrows':int(len(best_params)/4),'ncols':4,'figsize':(10,10)}
dda.plot_params([best_params],kwargs=kwargs_plot_params)

# produce output excel file with data, fitparameters and FOMs
fn_xlsx = 'fits_results.xlsx'
namecols = X_dimensions + ['Jexp','Jfit']
# delete old file if it exists
if os.path.exists(os.path.join(res_dir,fn_xlsx)):
    os.remove(os.path.join(res_dir,fn_xlsx))

with pd.ExcelWriter(os.path.join(res_dir,fn_xlsx), mode='w') as writer:
    df = pd.DataFrame(data,columns=namecols)
    df.to_excel(writer, sheet_name = 'data') 
    df = pd.DataFrame(pout,columns=[k for k in param_dict[0].keys()])
    df.to_excel(writer, sheet_name = 'params') 

In [ ]:
# Send fittings results to slack
send2slack = False
if send2slack:
    # send JV fits to slack
    filename = os.path.join(kwargs_JV_plot['figdir'],kwargs_JV_plot['figname']+'.png')
    send_file(filename,channel='#boar',message=f'JV fits deice PID {PID}')

In [ ]:
# Clean output files from simulation folders
from boar.SIMsalabim_utils.CleanFolder import *
Do_Cleaning = True # Careful, this will delete all files in the folder
#path2simu =res_dir
if Do_Cleaning:
    clean_up_output('tj',path2simu)
    clean_up_output('tVG',path2simu)
    clean_up_output('JV',path2simu)
    clean_up_output('Var',path2simu)
    clean_up_output('scPars',path2simu)
    clean_up_output('Str4Parallel',path2simu)
    clean_up_output('device_parameters_',path2simu)
    clean_up_output('logjob',path2simu)
    # os.remove(mo.path2oldxy) # remove the old_xy.json file if it exists
    # delete warmstart folder if it exists
    # if os.path.exists(os.path.join(os.getcwd(),'warmstart/')):
    #     shutil.rmtree(os.path.join(os.getcwd(),'warmstart/'))
    # delete temp folder if it exists
    # if os.path.exists(os.path.join(os.getcwd(),'temp/')):
    #     shutil.rmtree(os.path.join(os.getcwd(),'temp/'))

